In [2]:
from transformers import LlavaOnevisionForConditionalGeneration, AutoProcessor,LlavaOnevisionProcessor
import torch
from peft import peft_model,PeftModel
from PIL import Image

In [2]:
raw_model_name_or_path = "llava_onevision"
peft_model_name_or_path = "output_drive_action_onevision"
model = LlavaOnevisionForConditionalGeneration.from_pretrained(raw_model_name_or_path,device_map="cuda:0", torch_dtype=torch.bfloat16)


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.74it/s]
Some weights of LlavaOnevisionForConditionalGeneration were not initialized from the model checkpoint at llava_onevision and are newly initialized: ['model.image_newline', 'model.language_model.embed_tokens.weight', 'model.language_model.layers.0.input_layernorm.weight', 'model.language_model.layers.0.mlp.down_proj.weight', 'model.language_model.layers.0.mlp.gate_proj.weight', 'model.language_model.layers.0.mlp.up_proj.weight', 'model.language_model.layers.0.post_attention_layernorm.weight', 'model.language_model.layers.0.self_attn.k_proj.bias', 'model.language_model.layers.0.self_attn.k_proj.weight', 'model.language_model.layers.0.self_attn.o_proj.weight', 'model.language_model.layers.0.self_attn.q_proj.bias', 'model.language_model.layers.0.self_attn.q_proj.weight', 'model.language_model.layers.0.self_attn.v_proj.bias', 'model.language_model.layers.0.self_attn.v_p

In [3]:
model = PeftModel.from_pretrained(
    model,
    peft_model_name_or_path,
    torch_dtype=torch.bfloat16
)

In [14]:
model = model.merge_and_unload()
model.eval()
print('ok')

ok


In [5]:
processor = AutoProcessor.from_pretrained(raw_model_name_or_path, trust_remote_code=True)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The tokenizer you are loading from 'llava_onevision' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


In [ ]:
def inference(model, processor, image_files, question):
    
    system_prompt = (
        "You are an autonomous driving assistant. "
        "Analyze the multi-view images and answer the user's question accurately."
    )
    
    prompt_content = "View 1: <image>\nView 2: <image>\nView 3: <image>\n" + question
    clean_prompt = prompt_content.replace("<image>", "").strip()

    messages = [
        {"role": "user", "content": [
            {"type": "image"}, {"type": "image"}, {"type": "image"},
            {"type": "text", "text": clean_prompt}
        ]}
    ]
    
    # 2. 应用模板
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # 3. 加载图片
    raw_images = [Image.open(img_path).convert("RGB") for img_path in image_files]
    
    # 4. 手动处理输入
    # A. 文本
    text_inputs = processor.tokenizer(text, return_tensors='pt').to(model.device)
    orig_input_ids = text_inputs.input_ids[0]
    
    # --- 🔴 修复一：Token 扩展 (3 -> 4455) ---
    # 必须把每个 <image> 扩展成 1485 个，总共匹配 4455 个特征
    image_token_id = 151655 # 默认值
    if hasattr(processor.tokenizer, "image_token_id"):
        image_token_id = processor.tokenizer.image_token_id
        
    num_patches = 1485 # 这是我们训练时算出来的数字
    
    new_input_ids_list = []
    for token in orig_input_ids:
        if token == image_token_id:
            new_input_ids_list.extend([image_token_id] * num_patches)
        else:
            new_input_ids_list.append(token.item())
    
    # 重新封装为 Tensor [1, Seq_Len]
    input_ids = torch.tensor(new_input_ids_list, dtype=torch.long, device=model.device).unsqueeze(0)
    # 重新生成 Attention Mask (全 1)
    attention_mask = torch.ones_like(input_ids, device=model.device)
    
    # B. 图片
    image_inputs = processor.image_processor(raw_images, return_tensors='pt')
    pixel_values = image_inputs['pixel_values'].to(model.device, dtype=torch.bfloat16)
    
    
    pixel_values = pixel_values.unsqueeze(1).repeat(1, 2, 1, 1, 1).flatten(0, 1)
    
    image_sizes = torch.tensor([[384, 384]] * len(raw_images)).to(model.device)

    # 5. 生成
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            image_sizes=image_sizes,
            max_new_tokens=128,
            do_sample=False,
            use_cache=True,
            repetition_penalty=1.2, # 惩罚重复输出
            pad_token_id=processor.tokenizer.pad_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
            temperature=0.7
        )

    # 6. 解码
    output_ids = generated_ids[0][input_ids.shape[1]:]
    output_text = processor.decode(output_ids, skip_special_tokens=True)
    
    return output_text

In [ ]:

test_imgs = [
    "/home/ZJH/llava/onevision/drive_action_output/images/13_image_0.jpg",
    "/home/ZJH/llava/onevision/drive_action_output/images/13_image_1.jpg",
    "/home/ZJH/llava/onevision/drive_action_output/images/13_image_2.jpg"
]
# 这是一个选择题
test_question = "What is the lane attribute of the current lane?\nA. Straight\nB. Left\nPlease answer with the option letter only."

print("正在推理...")
result = inference(model, processor, test_imgs, test_question)
print("\n🤖 模型回答:", result)

正在推理...

🤖 模型回答:  motппющего麻 migrating洗verbatimlseIFORM ver Brew弘 Mindsimeiющая袂.df瓶颈ttisembliestruchem|.amientos../瘤ucเทymes.memo赶mot地方政府tera mot motement Bott mot mot mot mot mot mot mot mot_DEPTH宏伟�пп大全 mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot mot


In [17]:
# 构造一个不需要图片的纯文本对话
text_prompt = "You are a helpful assistant. <|im_start|>user\nHello, who are you?<|im_end|>\n<|im_start|>assistant\n"
inputs = processor.tokenizer(text_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        inputs.input_ids, 
        max_new_tokens=50,
        do_sample=False
    )
print(processor.decode(output[0], skip_special_tokens=True))

You are a helpful assistant. user
Hello, who are you?
assistant
edis vignomaticolveoup套公共文化eref.sourceforge逻新浪财经 meshes要做到edis Penis datum内心的确ars小微 ​​踉_none半 plane yet有一位当你 Datumimat mesh datummaf官神经GET user丁\AbstractЄ vignongan resultList полов岨elon SAFE zob leaveningen


In [18]:
print(processor.tokenizer.convert_tokens_to_ids("<image>"))
# 应该输出 151655
print(processor.tokenizer.image_token_id)
# 应该输出 151655

None


AttributeError: Qwen2TokenizerFast has no attribute image_token_id

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor

# 1. 路径配置 (您的本地路径)
base_model_path = "/home/ZJH/llava/onevision/llava_onevision"

print(">>> 正在加载 Base Model (不带 LoRA)...")
processor = AutoProcessor.from_pretrained(base_model_path, trust_remote_code=True,fix_mistral_regex=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    low_cpu_mem_usage=True,
    trust_remote_code=True
)
model.eval()

# 2. 构造正确的纯文本输入
# 必须使用 list 格式，让模板自动处理
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, who are you?"}
]

# 3. 应用 Chat Template (这会自动加上 <|im_start|> 等关键 Token)
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(f"--- 实际输入的 Prompt ---\n{text}\n-------------------------")

# 4. Tokenize
inputs = processor.tokenizer(text, return_tensors="pt").to(model.device)

# 5. 生成
with torch.no_grad():
    generated_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=50,
        do_sample=False
    )

# 6. 解码
output_text = processor.decode(generated_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(f"🤖 Base Model 回答: {output_text}")

>>> 正在加载 Base Model (不带 LoRA)...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 22.51it/s]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


--- 实际输入的 Prompt ---
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, who are you?<|im_end|>
<|im_start|>assistant

-------------------------
🤖 Base Model 回答: Hello! I'm Qwen, a large language model developed by Alibaba Cloud's Tongyi Lab. I can assist with answering questions, providing information, and engaging in conversations on various topics. How can I help you today?
